In [1]:
from IPython.display import IFrame

import numpy as np
import pandas as pd
import pickle

import config
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

In [3]:
def load(filename = "filename.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
        
    except FileNotFoundError: 
        print("File not found!")

In [4]:
def display_artist(track_info):
## create a single string with all artist names
    return " / ".join([x['name'] for x in track_info['artists']])

In [5]:
def create_artist_input_prompt(artist_names):
    user_prompt = ""
    valid_input = []
    
    for i in range(len(artist_names)):
        my_num = i
        user_prompt += f"Enter {my_num} for '{artist_names[i]}'\n"
        valid_input.append(str(my_num))
    ## space is always none of the above
    user_prompt += "Enter [SPACE] for 'None of the above'\n"
    valid_input.append(" ")
    
    return valid_input, user_prompt

In [6]:
def recommend_hot_song(hot_songs_df, not_this_title):
    myrec = hot_songs_df.sample()
    
    if (myrec.title == not_this_title).all():
        print("DEBUG: try again")
        myrec = hot_songs_df.sample()
    return myrec

In [8]:
## Load hot songs df
infile_list = ['Data/db/hot/billboard_hot100_2021-02-14.csv', 
               'Data/db/hot/uk_hot100_2021-02-14.csv', 
               'Data/db/hot/hot_songs_miri.csv']

dfs = []
for fname in infile_list:
    dfs.append(pd.read_csv(fname, index_col=None))
    
hot_songs_df = pd.concat(dfs, axis=0)
hot_songs_df = hot_songs_df.drop_duplicates().reset_index(drop=True)